# Training Mobile Net For Object Detection 

In this notebook, we will give you some fundamental steps to Train an Object Detection Model using Tensorflow Object Detection API.
You don't need to have complete knowledge of neural networks and machine learning for using this API, as we are mostly going to use the files provided by the Tensorflow Object Detection API. All you need to have is knowledge of python and know how to work on a jupyter notebook.

## Setup colab Working directory (Google Colab only)

After uploading the *ml-usecase-tensorflow-object-detection* file to your Google Drive and open this notebook you need to mount the drive and setup a working directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

%cd drive/MyDrive/<'path to the ml-usecase-tensorflow-object-detection file'>

## Setup Workspace

In [ ]:
import os
import cv2 
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'  # Name of the Network we are going to use 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('MyFirstTFOD','Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('MyFirstTFOD','Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('MyFirstTFOD','Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Download TensorFlow Model Garden 
In this library, we will have access to the Tensorflow Object Detection API.
Under the models/research directory you will find the code implementations and pre-trained models of published research papers.

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

## Download and Install Protobuf
Protocol Buffers known as *protobuf* are Google's language-neutral, platform-neutral, extensible mechanism for serializing structured data.
If the code below fails install the compiler follow this [link](https://github.com/protocolbuffers/protobuf).

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    #!brew install protobuf-c # this is for mac os distro
    !apt-get install protobuf-compiler # this is for linux distro
    !cd MyFirstTFOD/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd MyFirstTFOD/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd MyFirstTFOD/Tensorflow/models/research/slim && pip install -e .

## Verify Build

This build test allows you to check if everything is **ready to go**. Just wait until it prints :

    ``OK (skipped=1)``

    

### If the verification code returns:

``Error: No module named ´<module-name>´``, just **install** the module by running ``!pip install ‘<module-name>’`` in a cell.

or

``Error: ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject``, you need to **Reinstall pycocotools** by running``!pip uninstall pycocotools -y`` and  ``!pip install pycocotools`` in a cell.

or

``Error: AttributeError: module 'sip' has no attribute 'setapi'``, you need to **Downgrade matplotlib** by running ``!pip uninstall protobuf matplotlib -y`` and ``!pip install protobuf matplotlib==3.2`` in a cell

### To Run with GPU 
If you want to run using GPU you can just make sure you have the **right** CUDA and CUNN **versions installed** ([Windows](https://www.tensorflow.org/install/source_windows), [Linux/macOs](https://www.tensorflow.org/install/source))

**Note: If you are using Google Colab you don't need to install CUDA**

Then **install** tensorflow-gpu ``!pip install tensorflow-gpu`` in a cell.

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

## Import Object Detection 

In [ ]:
import object_detection

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

## Create Label Map

Here we are creating the labels of our image set so the model can recognize them.

    Note: It's important to give the same 'name' as the one given in the LabelImg annotation.

In [ ]:
labels = [{'name':'ThumbsUp', 'id':1}, {'name':'ThumbsDown', 'id':2}, 
                {'name':'Peace', 'id':3}, {'name':'ThankYou', 'id':4}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Create TF records
TFRecord is an important data format designed for Tensorflow. Before you can train your custom object detector, you must convert your data into the TFRecord format.
For this, we are going to use a python script available in the following Git Repository * https://github.com/nicknochnack/GenerateTFRecord*.

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

## Model Configuration 

The pre-trained model is present in the models/research directory of Tensorflow as a configuration file where all the weights and parameters are defined.

This configuration file allows you to use **Transfer Learning** that enables the model to run properly on any device.

To be able to use it you need to configure the paths in this file. The code below does that for you. 

In [ ]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Copy the Model configuration to workspace
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

## Train the Model

This step can be done locally, but be aware that it can take a while. 
If you are using Google Colab make sure you have the GPU enabled. 



**Note: Before executing these next cells you need to **verify the OpenCv dependencies**.**

All of them need to be in the *same version* especially the *opencv-python* and *opencv-python-headless* (We tested in version 4.1.2.30).

Run the cell below to verify dependencies.

In [ ]:
!pip list | grep opencv

If *opencv-python* and *opencv-python-headless* are in *different versions*, run in a cell:

``!pip uninstall opencv-python-headless``

``!pip install opencv-python-headless==4.1.2.30``

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
!{command}

## Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

## Load Trained Model 
Now we are going to retrieve the trained model from the last checkpoint and perform detection on a single image.

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(tensor):
    image, shapes = detection_model.preprocess(tensor)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'thumbsdown.adbed59c-9fa2-11ec-940d-acde48001122.jpg')

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

## Zip and Export Models (Colab Only)
If you are using colab to execute this training notebook, please zip it and export it to your working directory.

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}